#  Code for 2D staggered Lagrangian method in Cartersian coordinates

## Control equations

### Momentum equation

$$\tag{1.1}\rho \frac{d\mathbf{v}}{dt} =  -\nabla p $$ 
where $\mathbf{v}$ is the vector of velocity, and $\rho$ is the density and $p$ is the pressure.  

### Energy equation

$$ \rho \frac{dE}{dt} = -\nabla \cdot (p\mathbf{v})$$ 

where $E$ is the specific total energy and it has a realtion with the specific internal energy 
$$E = e + \frac{1}{2}\mathbf{v}\cdot\mathbf{v}$$

By (1.1) and (1.2) we have 
$$\tag{1.3} \rho \frac{de}{dt} = - p\nabla\cdot \mathbf{v}$$

In  a Cartersian coordinates $x-y$, The equations (1.1) and (1.3) can be written as
$$\rho \frac{du}{dt} = -\frac{\partial p}{\partial x},\quad \rho \frac{dv}{dt} = -\frac{\partial p}{\partial y}$$
and
$$\rho \frac{de}{dt} = -p (\frac{\partial u}{\partial x} +\frac{\partial v}{\partial y})$$

## Compatible Staggered Discretization

<img src="Grid.png" width = "800" height = "400" div align=center /> 

### Quatities:

On point: ($x_p$,$y_p$), ($u_p$,$v_p$)

Zonal:  $V_z$, $\rho_z$, $m_z$, $e_z$, $p_z$

By the relation 
$$\frac{dm_z}{dt} =0$$
we have 
$$\rho_z(t) = m_z/V_z(t)$$

Also 
$$d(x_p)/dt = u_p, \quad d(y_p)/dt = v_p$$

### Point mass 

Point mass $m_p$ is used to solve point momentum $\mu_p = m_p \bf{u}_p$ and kinetic energy $K_p = m_p \frac{\bf{u}_p^2}{2}$

<img src="Ins.jpg" width = "200" height = "400" div align=center /> 

$$m_p = \sum_{z\in Z(p)} A_z^p \rho^p_z$$

$$\rho_z(t) = m_z/V_z(t)$$

In [1]:
function MassAndDens(mz::Array{Float64,1},x::Array{Float64,1},y::Array{Float64,1})
    
    Ic, = size(mz)
    Ip, = size(Tc)
    Asbz = zeros(Float64,4)
    ρ = zeros(Float64,Ic)
    mp = zeros(Float64,Ip)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    for i in 1:Ic
        xz[1:4] = x[Tp[i,1:4]]
        yz[1:4] = y[Tp[i,1:4]]
        Asbz = A_z(i,xz,yz)
        Az = sum(Asbz[j] for j in 1:4)
        ρ[i] = mz[i]/Az
        
        for j = 1:4
            ip = Tp[i,j]
            mp[ip] = mp[ip] + Asbz[j]*ρ[i]
        end
    end
    return ρ,mp
end           
        

MassAndDens (generic function with 1 method)

In [2]:
mz

UndefVarError: UndefVarError: mz not defined


$$ A_z^1 = \frac{5A_{41}+5A_{12}+A_{23}+A_{34}}{12}$$
$$ A_z^2 = \frac{A_{41}+5A_{12}+5A_{23}+A_{34}}{12}$$
$$ A_z^3 = \frac{A_{41}+A_{12}+5A_{23}+5A_{34}}{12}$$
$$ A_z^4 = \frac{5A_{41}+A_{12}+A_{23}+5A_{34}}{12}$$

In [3]:
function A_z(ic::Int,x::Array{Float64,1},y::Array{Float64,1}) 
    A =zeros(Float64,4)
    xc= sum(x[i] for i in 1:4)/4
    yc= sum(y[i] for i in 1:4)/4
    
    A41 = Area3(x[1],x[4],xc,y[1],y[4],yc)
    A12 = Area3(x[1],x[2],xc,y[1],y[4],yc)
    A23 = Area3(x[2],x[3],xc,y[2],y[3],yc)
    A34 = Area3(x[3],x[4],xc,y[3],y[4],yc)
    
    A[1] = (5A41+5A12+A23+A34)/12
    A[2] = (A41+5A12+5A23+A34)/12
    A[3] = (A41+A12+5A23+5A34)/12
    A[4] = (5A41+A12+A23+5A34)/12
    
    return A
end

A_z (generic function with 1 method)

<img src="Ins2.jpg" width = "400" height = "300" div align=center />

In [4]:
function F_z(x::Array{Float64,1},y::Array{Float64,1},p::Float64)
   
    Fx = zeros(Float64,4)
    Fy = zeros(Float64,4)
    
    L12 = √((x[2]-x[1])^2+(y[2]-y[1])^2)
    n12x = (-y[2]+y[1])/L12
    n12y = (x[2]-x[1])/L12
    
    L41 = √((x[1]-x[4])^2+(y[1]-y[4])^2)
    n41x = (-y[1]+y[4])/L41
    n41y = (x[1]-x[4])/L41
    
    L23 = √((x[3]-x[2])^2+(y[3]-y[2])^2)
    n23x = (-y[3]+y[2])/L23
    n23y = (x[3]-x[2])/L23
    
    L34 = √((x[4]-x[3])^2+(y[4]-y[3])^2)
    n34x = (-y[4]+y[3])/L34
    n34y = (x[4]-x[3])/L34
    
    Fx[1] = p*(L12*n12x+L41*n41x)/2
    Fy[1] = p*(L12*n12y+L41*n41y)/2
    
    Fx[2] = p*(L12*n12x+L23*n23x)/2
    Fy[2] = p*(L12*n12y+L23*n23y)/2
    
    Fx[3] = p*(L23*n23x+L34*n34x)/2
    Fy[3] = p*(L23*n23y+L34*n34y)/2
    
    Fx[4] = p*(L34*n34x+L41*n41x)/2
    Fy[4] = p*(L34*n34y+L41*n41y)/2
    
    return Fx,Fy
    end           

F_z (generic function with 1 method)

对于单元 
$$ m_z \frac{de_z}{dt} = -\sum_{z\in Z(p)} \bf{f}_z^p \cdot \bf{u}_p $$  

对于节点
$$m_p \frac{d\bf{u}_p}{dt} = \sum_{p\in P(z)} \bf{f}_z^p$$

In [17]:
function Rhs(x::Array{Float64,1},y::Array{Float64,1},u::Array{Float64,1},v::Array{Float64,1},p::Array{Float64,1})  #Right hand e
    Ic, = size(Tp)
    Ip, = size(Tc)
    rhsu = zeros(Float64,Ip)
    rhsv = zeros(Float64,Ip) 
    rhse = zeros(Float64,Ic)
    xz = zeros(Float64,4)
    yz = zeros(Float64,4)
    
    for i in 1:Ic
        for j =1:4
            xz[j] = x[Tp[i,j]]
            yz[j] = y[Tp[i,j]]
        end
        Fx,Fy = F_z(xz,yz,p[i])
        
        ∑=0
        for j in 1:4
            ip = Tp[i,j]
            ∑ = ∑ +u[ip]*Fx[j] + v[ip]*Fy[j]
        end
        rhse[i] = -∑
        
        for j = 1:4
            ip = Tp[i,j]
            rhsu[ip] = rhsu[ip] + Fx[j]
            rhsv[ip] = rhsv[ip] + Fy[j]
        end     
    end
    return rhse, rhsu,rhsv
end

Rhs (generic function with 1 method)

In [22]:
function Euler1st(dt,u,v,x,y,p,mz,ρ)
    Ip, = size(u)
    Ic, = size(p)
    
    ρ,mp = MassAndDens(mz,x,y)
    
    e = pToe(p, ρ,problem.γ)
    rhse, rhsu,rhsv = Rhs(x,y,u, v, p)
    
    rhsu,rhsv = Bound(x,y,u, v, p)
    u  += dt*(rhsu ./ mp)
    v  += dt*(rhsv ./ mp)
    e  += dt*(rhse ./ mz)
    
    x  +=  dt*u
    y  +=  dt*v
    
    ρ,mp = MassAndDens(mz,x,y)
    
    p = eTop(e, ρ,problem.γ)
    return u,v,x,y,p,ρ
end

Euler1st (generic function with 2 methods)

### EOS 
$$ e = \frac{p}{(\gamma-1)\rho}$$

In [6]:
function pToe(p,ρ,γ)
    return p ./ ρ/(γ-1)
end
function eTop(e,ρ,γ)
    return (γ-1)*ρ .* e
end

eTop (generic function with 1 method)

In [7]:
function CFL(SF,x::Array{Float64,1},y::Array{Float64,1},
                u::Array{Float64,1},v::Array{Float64},
                p::Array{Float64,1},ρ::Array{Float64,1})
    Ic, = size(ρ)
    cflmin = 1.e9
    cfl =0
    γ = problem.γ
    for i in 1:Ic
        u2max = 0
        u2=0.0
        for j = 1:4
            u2 = u[j]^2+v[j]^2
            if u2 > u2max ; u2max = u2; end
        end
        
        dlmin = 1.e9
        for j=1:3
            dl= (x[j+1] - x[j])^2+(y[j+1]-y[j])^2
            if dl < dlmin; dlmin=dl; end
        end
        c = √(γ*p[i]/ρ[i])
        cfl = √(dlmin)/(√(u2)+c)
        if cfl < cflmin; cflmin = cfl; end
    end
    dt = cfl*SF
    return dt
end 
        

CFL (generic function with 1 method)

In [25]:
function TimeSolve(u,v,x,y,p,mz,ρ)
    tt= problem.tt
    sf =problem.sf
    t= 0.0
#    while t<tt
for i in 1:2
        dt=CFL(sf,x,y,u,v,p,ρ)
        if t+dt>tt
           dt = tt-t
        end
        
        u,v,x,y,p,ρ = Euler1st(dt,u,v,x,y,p,mz,ρ)
    t += dt
        println(t)
    end
#
    return u,v,x,y,p,ρ
end

TimeSolve (generic function with 1 method)

In [26]:
x,y,u,v,mz,p,ρ,Az=InitTest1()
u,v,x,y,p,ρ = TimeSolve(u, v, x, y, p, mz,ρ)

0.0012677313820927745
0.001474106258247412


([-1.96749, -0.380653, 3.80629e-14, -1.83871e-14, -1.51221e-14, 6.69323e-14, -7.00255e-14, -2.44015e-14, -1.86448e-14, 3.71178e-14  …  1.30058e-12, -2.73636e-13, 2.97565e-13, -1.15662e-12, -6.73469e-13, 8.29866e-13, -2.17745e-13, 1.41898e-13, 0.00729655, -0.791689], [-2.31353, 6.63338, 4.26032, 4.26032, 4.26032, 4.26032, 4.26032, 4.26032, 4.26032, 4.26032  …  -1.50356, -1.50356, -1.50356, -1.50356, -1.50356, -1.50356, -1.50356, -1.50356, -1.4581, -0.806425], [0.00202253, 0.00992144, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05  …  0.96, 0.965, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 1.0, 1.00398], [0.00195112, 0.0140832, 0.0135935, 0.0135935, 0.0135935, 0.0135935, 0.0135935, 0.0135935, 0.0135935, 0.0135935  …  0.103147, 0.103147, 0.103147, 0.103147, 0.103147, 0.103147, 0.103147, 0.103147, 0.103156, 0.103976], [-0.353184, -2.41522, -2.48847, -2.48847, -2.48847, -2.48847, -2.48847, -2.48847, -2.48847, -2.48847  …  0.141102, 0.141102, 0.141102, 0.141102, 0.141102, 0.141102, 0.141102,

In [37]:
using Plots
plots()    

UndefVarError: UndefVarError: plots not defined

In [1]:
Ip, = size(Tc) 
p = zeros(Float64,Ip,4)
for i in 1:Ip
    p[i,1] = x[i]
    p[i,2] = y[i]
    p[i,3] = u[i]
    p[i,4] = v[i]
end
p

UndefVarError: UndefVarError: Tc not defined

In [2]:
scatter3D(p[:,1],p[:,2],p[:,3])

UndefVarError: UndefVarError: p not defined

In [9]:
function InitTest1()

    global problem = prb(0.1,0.3,1.4)
     
    I =200
    J =20
    dx = 1.0/I
    dy = 0.1/J
    
    
    Ip = (I+1)*(J+1) # Number of  points
    Ic = I*J  # number of cells
    
    global Tp = zeros(Int,Ic,4) #格点
    global Tc = zeros(Int,Ip,4) #点格
    
    IBL = J+1
    IBR = J+1
    IBU = I+1
    IBD = I+1
   # 点格表，格点表初始化 
    for i in 1:200 
        for j in 1:20
            ic = i+(j-1)*200
            Tp[ic,1] = i + (j-1)*201
            Tp[ic,2] = i+1 +(j-1)*201
            Tp[ic,3]=i+1+j*201
            Tp[ic,4]=i+j*201
        end
    end
    
    
    for i in 1:201
        for j in 1:21
            ip = i+(j-1)*201
            Tc[ip,1] = i +(j-1)*200
            if i==1 
                Tc[ip,2] = 0  
            else
                Tc[ip,2] = i-1+(j-1)*200
            end
            if i==1 & j ==1
                Tc[ip,3] = 0
            else
                Tc[ip,3] = i-1+(j-2)*200
            end
            
            if j==1
                Tc[ip,4] = 0
            else
                Tc[ip,4] = i+(j-2)*200
            end
        end
    end
    
    #流场初始化 Init of the flow
    
    x = zeros(Float64,Ip)
    y = zeros(Float64,Ip)
    u = zeros(Float64,Ip)
    v = zeros(Float64,Ip)
    
    ρ = zeros(Float64,Ic)
    mz = zeros(Float64,Ic)
    p = zeros(Float64,Ic)
    Az= zeros(Float64,Ic)
    for i in 1:201
        for j = 1:21
            ip =i+(j-1)*201
            x[ip] = i*dx
            y[ip] = j*dy
            
            u[ip] = 0.0
            v[ip] = 0.0
        end
    end
    
    xz = zeros(Float64,4)
    yz = zeros(Float64,4) 
    for i in 1:200
        for j in 1:20
            ic = i+(j-1)*200
            xz[1:4] = x[Tp[ic,1:4]]
            yz[1:4] = y[Tp[ic,1:4]]
            A = A_z(ic,xz,yz)
            Az[ic] = A[1]+A[2]+A[3]+A[4]
            if i<=100
                mz[ic] = 1.0* Az[ic]
                p[ic] = 1.0
                ρ[ic] = 1.0
            else
                mz[ic] = 0.1 * Az[ic]
                p[ic] = 0.1
                ρ[ic] = 0.1
            end
        end
    end
    
    return x,y,u,v,mz,p,ρ,Az
end

InitTest1 (generic function with 1 method)

In [10]:
struct prb
    tt::Float64
    sf::Float64
    γ ::Float64
end

In [11]:
function Area3(x1,x2,x3,y1,y2,y3)
    return (x1*y2+y1*x3+x2*y3-x1*y3-y1*x2-y2*x3)/2
end

Area3 (generic function with 1 method)

In [34]:
using BenchmarkTools
@btime sum(i for i in 1:4)

  2.515 ns (0 allocations: 0 bytes)


10

In [50]:
A = (3,2,2)

(3, 2, 2)

In [51]:
typeof(A)

Tuple{Int64,Int64,Int64}

In [52]:
A ::Tuple

(3, 2, 2)

In [53]:
A[1] = (3,2,2)

MethodError: MethodError: no method matching setindex!(::Tuple{Int64,Int64,Int64}, ::Tuple{Int64,Int64,Int64}, ::Int64)

BoundsError: BoundsError

In [70]:
? append!

search: append!



```
append!(collection, collection2) -> collection.
```

Add the elements of `collection2` to the end of `collection`.

# Examples

```jldoctest
julia> append!([1],[2,3])
3-element Array{Int64,1}:
 1
 2
 3

julia> append!([1, 2, 3], [4, 5, 6])
6-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
```

Use [`push!`](@ref) to add individual items to `collection` which are not already themselves in another collection. The result of the preceding example is equivalent to `push!([1, 2, 3], 4, 5, 6)`.

---

```
append!(cb, datavec)
```

Push at most last `capacity` items.


In [63]:
B = [1; 2; 3; 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [68]:
? BitArray

search: BitArray



```
BitArray{N} <: AbstractArray{Bool, N}
```

Space-efficient `N`-dimensional boolean array, using just one bit for each boolean value.

`BitArray`s pack up to 64 values into every 8 bytes, resulting in an 8x space efficiency over `Array{Bool, N}` and allowing some operations to work on 64 values at once.

By default, Julia returns `BitArrays` from [broadcasting](@ref Broadcasting) operations that generate boolean elements (including dotted-comparisons like `.==`) as well as from the functions [`trues`](@ref) and [`falses`](@ref).

---

```
BitArray(undef, dims::Integer...)
BitArray{N}(undef, dims::NTuple{N,Int})
```

Construct an undef [`BitArray`](@ref) with the given dimensions. Behaves identically to the [`Array`](@ref) constructor. See [`undef`](@ref).

# Examples

```julia-repl
julia> BitArray(undef, 2, 2)
2×2 BitArray{2}:
 false  false
 false  true

julia> BitArray(undef, (3, 1))
3×1 BitArray{2}:
 false
 true
 false
```

---

```
BitArray(itr)
```

Construct a [`BitArray`](@ref) generated by the given iterable object. The shape is inferred from the `itr` object.

# Examples

```jldoctest
julia> BitArray([1 0; 0 1])
2×2 BitArray{2}:
  true  false
 false   true

julia> BitArray(x+y == 3 for x = 1:2, y = 1:3)
2×3 BitArray{2}:
 false   true  false
  true  false  false

julia> BitArray(x+y == 3 for x = 1:2 for y = 1:3)
6-element BitArray{1}:
 false
  true
 false
  true
 false
 false
```
